In [2]:
import pandas as pd
import glob
from google.cloud import bigquery
%load_ext google.cloud.bigquery

In [3]:
data_folder = '/home/reza/patstat2018/'
file_2 = 'exact_twins_docdb-*.csv.gz'
file_1 = 'family_patents/data/familyInformation.csv.zip'

# 1. Loading datasets:

## 1.1. `df_twins_1` (the backbone dataset which exists in "family_information" bucket)

In [13]:
# the dataframe that from 'familyInformation'
df_twins_1 = pd.read_csv(data_folder+file_1, compression='zip', 
                         sep=';', usecols=[0,1,7,12,13], index_col=None)

print('There are {:,} number of rows in df_twins_1!\n'.format(df_twins_1.shape[0]))
df_twins_1.head()

There are 17,218,989 number of rows in df_twins_1!



,docdb_family_id,appln_id_1,appln_id_2,family_size_in_auth2,exact_twins
0,8164667,448453,267441867,1,1
1,8164667,448453,267566903,1,1
2,39810287,448454,448456,12,0
3,39810287,448454,449293,12,0
4,39810287,448454,449431,12,0


In [16]:
#df_twins_1.to_csv(data_folder + 'family_patents/data/'+ 'familyInformation.csv.gz', compression='gzip', index=False)

In [14]:
df_twins_1[df_twins_1.appln_id_2==412207741]

,docdb_family_id,appln_id_1,appln_id_2,family_size_in_auth2,exact_twins
1926111,22894414,48879223,412207741,1,1


## 1.2. `df_twins_2` (the backbone dataset which I created by query)

In [20]:
allFiles = glob.glob(data_folder+file_2)

allFiles

['/home/reza/patstat2018/exact_twins_docdb-000000000000.csv.gz',
 '/home/reza/patstat2018/exact_twins_docdb-000000000002.csv.gz',
 '/home/reza/patstat2018/exact_twins_docdb-000000000003.csv.gz',
 '/home/reza/patstat2018/exact_twins_docdb-000000000001.csv.gz']

In [17]:
columns_list = list(pd.read_csv(allFiles[0], compression='gzip', nrows=10).columns)
df_twins_2 = pd.DataFrame(columns=columns_list)

df_twins_2.head()

,appln_id_1,appln_nr_1,appln_auth_1,appln_filing_year_1,appln_id_2,appln_nr_2,appln_auth_2,appln_filing_year_2,docdb_family_id,docdb_nb_pairs,exact_twins


In [22]:
list_ = []

for file_ in allFiles:
    df_twins_2 = pd.concat([df_twins_2,pd.read_csv(file_, compression='gzip', low_memory=False)], axis=0, ignore_index=True)

# Dropping pairs which either of them filed in 2018
df_twins_2 = df_twins_2[(df_twins_2.appln_filing_year_1!=2018)&(df_twins_2.appln_filing_year_2!=2018)]
print('There are {:,} number of rows in df_twins_2!\n'.format(df_twins_2.shape[0]))
df_twins_2.head()

In [87]:
df_twins_2.appln_filing_year_1.unique()

array([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017], dtype=object)

In [65]:
#df_difference = pd.merge(df_twins_1, df_twins_2, how='outer', indicator=True, on=['appln_id_1', 'appln_id_2'])\
#                  .query("_merge!='both'").drop('_merge', axis=1)

# 2. Exploring discrepencies in `"df_twins_1"` and `"df_twins_2"`

## 2.1. Getting the records that are not mutual between `df_twins_1` and `df_twins_2`

In [ ]:
client = bigquery.Client()

job_config = bigquery.QueryJobConfig()
job_config.user_query_cache = False

# Set destination table
dataset_id = 'results_docdb'
table_id = 'not_mutual'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

In [ ]:
sql = """
    SELECT 
            appln_id_1,
            appln_id_2,
            t1.is_in_1,
            t2.is_in_2,
            exact_twins_1,
            exact_twins_2
            
    FROM
            (
            SELECT
                    appln_id_1,
                    appln_id_2,
                    exact_twins AS exact_twins_1,
                    1 AS is_in_1
            FROM
                    `usptobias.results_docdb.familyInformation`
            ) AS t1
            
        FULL OUTER JOIN
            (
            SELECT
                    appln_id_1,
                    appln_id_2,
                    exact_twins AS exact_twins_2,
                    1 AS is_in_2
            FROM
                    `usptobias.results_docdb.USExactTwins`
            WHERE
                    appln_filing_year_1 <> 2018
                    AND appln_filing_year_2 <> 2018
            ) AS t2

        USING(appln_id_1, appln_id_2)
    WHERE
            (t1.is_in_1 IS NULL OR t2.is_in_2 IS NULL)
            
     """
# Defining the query
query_job = client.query(
    sql,
    # Location must match that of the dataset(s) referenced in the query.
    location='US',
    job_config=job_config)

In [ ]:
query_job.result()

## 2.2. Adding additional information to `'appln_id_1'` and `'appln_id_2'`

In [13]:
client = bigquery.Client()

job_config = bigquery.QueryJobConfig()
job_config.user_query_cache = False

# Set destination table
dataset_id = 'results_docdb'
table_id = 'not_mutual_info'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref

In [14]:
sql = """
    SELECT 
            t1.*,
            t2.appln_auth_1,
            t2.appln_kind_1,
            t2.appln_filing_year_1,
            t3.appln_auth_2,
            t3.appln_kind_2,
            t3.appln_filing_year_2,
            t2.docdb_family_id,
            t2.docdb_family_size
            
    FROM
            `usptobias.results_docdb.not_mutual` AS t1
    LEFT JOIN
            (
            SELECT
                    appln_id AS appln_id_1,
                    appln_auth AS appln_auth_1,
                    appln_kind AS appln_kind_1,
                    appln_filing_year AS appln_filing_year_1,
                    docdb_family_id,
                    docdb_family_size
            FROM
                    `usptobias.patstat.tls201`
            ) AS t2
            ON t1.appln_id_1 = t2.appln_id_1
    LEFT JOIN
            (
            SELECT
                    appln_id AS appln_id_2,
                    appln_auth AS appln_auth_2,
                    appln_kind AS appln_kind_2,
                    appln_filing_year AS appln_filing_year_2
            FROM
                    `usptobias.patstat.tls201`
            ) AS t3
            ON t1.appln_id_2 = t3.appln_id_2
     """
# Defining the query
query_job = client.query(
    sql,
    # Location must match that of the dataset(s) referenced in the query.
    location='US',
    job_config=job_config)

In [15]:
query_job.result()

## 2.3. Exporting the Bigquery table to CSV file

In [16]:
# from google.cloud import bigquery
client = bigquery.Client()
bucket_name = 'patent-results'
folder_name = 'exactTwins'

project = 'usptobias'
dataset_id = 'results_docdb'
table_id = 'not_mutual_info'

destination_uri = 'gs://{}/{}/{}'.format(bucket_name, folder_name, 'not_mutual.csv.gz')
dataset_ref = client.dataset(dataset_id, project=project)
table_ref = dataset_ref.table(table_id)


job_config = bigquery.job.ExtractJobConfig()

job_config.compression = "GZIP"
job_config.destinationFormat = "CSV"

In [17]:
extract_job = client.extract_table(
    table_ref,
    destination_uri,
    # Location must match that of the source table.
    location='US',
    job_config=job_config)  # API request
extract_job.result()  # Waits for job to complete.

print('Exported {}:{}.{} to {}'.format(
    project, dataset_id, table_id, destination_uri))

Exported usptobias:results_docdb.not_mutual_info to gs://patent-results/exactTwins/not_mutual.csv.gz


# 3. Investigating the difference in datasets

In [128]:
df_diff = pd.read_csv(data_folder+'not_mutual.csv.gz', compression='gzip', index_col=None, header=0, low_memory=False)

print('There are {:,} rows in df_difference'.format(df_diff.shape[0]))
    
df_diff.head()

There are 4,472,433 rows in df_difference


,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size
0,53793286,5836350,1.0,NaN,0.0,NaN,US,A,2004.0,CH,A,1993.0,27571302.0,27.0
1,55523292,5836350,1.0,NaN,0.0,NaN,US,A,2008.0,CH,A,1993.0,27571302.0,27.0
2,54172971,18467090,1.0,NaN,0.0,NaN,US,A,2004.0,FI,A,1985.0,24868102.0,62.0
3,53196222,43704330,1.0,NaN,0.0,NaN,US,A,2003.0,SK,A,1993.0,9435406.0,48.0
4,53184814,43706513,1.0,NaN,1.0,NaN,US,A,2001.0,SK,A,1994.0,8089007.0,19.0


## 3.1. Looking at the pairs that exist only in `"df_twins_1"` (and not in `df_twins_2`)

In [129]:
# Getting rows in which 'df_twins_1' has an invalid appln_id
mask1_ = df_diff.is_in_1==1

df_ = df_diff[mask1_]
print('There are {:,} rows which only exist in "df_twins_1"!'.format(df_.shape[0]))

There are 2,255,994 rows which only exist in "df_twins_1"!


There are two problems:
1. There are some rows with invalid `appln_id`
2. There are some pairs which shouldn't be identified as exact twins!

For the **first problem**, we will take ta close look at each `appln_id_1` and `appln_id_2`.

For the **second problem**, I try to break down the problem into three cases:
1. Where the family size is `one` (where definitely should not be considered as exact twins)
2. Where the family size is `two` (where definitely should be considered as exact twins, if the second member has appln_auth in "US")
3. Where the family size is `more than two` (which depends on the situations)

### 3.1.1. Problem 1 : Some rows in "df_twins_1" don't have a valid `'appln_id'`

Investigating whether there is any invalid `appln_id_1` or `appln_id_2`:

In [92]:
# Getting rows in which 'df_twins_1' has an invalid appln_id
mask1_ = df_diff.is_in_1==1
mask2_ = df_diff.appln_auth_1.isna()
mask3_ = df_diff.appln_auth_2.isna()
df_invalid1_ = df_diff[(mask1_)&(mask2_)]
df_invalid2_ = df_diff[(mask1_)&(mask3_)]
df_invalid_ = df_diff[(mask1_)&((mask2_)|(mask3_))]
print('There are {:,} rows which has an invalid appln_id!'.format(df_invalid_.shape[0]))
display(df_invalid1_.head(3))
display(df_invalid2_.head(3))

There are 3,492 rows which has an invalid appln_id!


,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size
2249,449829386,450065768,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2532,438349221,275032931,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2619,450069160,449829386,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size
5,468013065,445750920,1.0,NaN,0.0,NaN,US,A,2016.0,NaN,NaN,NaN,47114625.0,9.0
7,54005182,3409763,1.0,NaN,0.0,NaN,US,A,2001.0,NaN,NaN,NaN,8553584.0,24.0
10,54310730,412209925,1.0,NaN,1.0,NaN,US,A,2007.0,NaN,NaN,NaN,18658401.0,4.0


Let's check for all invalid `appln_id`s that exist in `df_invalid1_` and `df_invalid2_` dataframe, to make sure that non of them exist in `tls201` dataset:

In [93]:
query = """
SELECT 
    appln_id, 
    appln_nr, 
    appln_filing_year, 
    appln_auth, 
    docdb_family_id, 
    docdb_family_size
FROM `usptobias.patstat.tls201`
WHERE appln_id IN UNNEST(@input_list);
"""
params = [
    bigquery.ArrayQueryParameter(
        'input_list', 'NUMERIC', list(df_invalid1_.appln_id_1)+list(df_invalid2_.appln_id_2))
]
job_config = bigquery.QueryJobConfig()
job_config.query_parameters = params

query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location='US',
    job_config=job_config)  # API request - starts the query

In [94]:
# Print the results
df_results_ = query_job.to_dataframe()
df_results_.head()

,appln_id,appln_nr,appln_filing_year,appln_auth,docdb_family_id,docdb_family_size


As we can see non of them exist in `tls_201` table, so they are invalid `appln_id`.
Thus, I remove those rows which doesn't make sense.

In [95]:
# Removing rows in which 'df_twins_1' has an invalid appln_id
df_diff = df_diff[~((df_diff.appln_auth_1.isna())|(df_diff.appln_auth_2.isna()))]

print('There are {:,} rows in df_difference'.format(df_diff.shape[0]))

df_diff.head(7)

There are 4,468,941 rows in df_difference


,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size
0,53793286,5836350,1.0,NaN,0.0,NaN,US,A,2004.0,CH,A,1993.0,27571302.0,27.0
1,55523292,5836350,1.0,NaN,0.0,NaN,US,A,2008.0,CH,A,1993.0,27571302.0,27.0
2,54172971,18467090,1.0,NaN,0.0,NaN,US,A,2004.0,FI,A,1985.0,24868102.0,62.0
3,53196222,43704330,1.0,NaN,0.0,NaN,US,A,2003.0,SK,A,1993.0,9435406.0,48.0
4,53184814,43706513,1.0,NaN,1.0,NaN,US,A,2001.0,SK,A,1994.0,8089007.0,19.0
6,54160629,20258943,1.0,NaN,0.0,NaN,US,A,2004.0,FR,A,1986.0,24868102.0,62.0
8,46553911,9008391,1.0,NaN,0.0,NaN,US,A,2002.0,CZ,A,1994.0,22551225.0,40.0


## 3.1.2. Problem 2 - Situation 1: family size of 1

In [96]:
# Getting a list of patents with family size of "one"
mask0_ = df_diff.is_in_1==1
mask1_ = df_diff.docdb_family_size==1
print('There are {:,} patents with family size of one!'.format(df_diff[(mask0_)&(mask1_)].shape[0]))
df_diff.sort_values(by=['docdb_family_size']).head()

There are 731 patents with family size of one!


,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size
3033439,451071091,456118537,1.0,NaN,0.0,NaN,US,A,2014.0,JP,A,2015.0,59011935.0,1.0
3988681,416694051,412771329,1.0,NaN,0.0,NaN,US,A,2013.0,US,A,2013.0,57847050.0,1.0
739703,365871272,471901769,1.0,NaN,1.0,NaN,US,A,2012.0,BR,A,2010.0,57868453.0,1.0
3876776,418647589,49603289,1.0,NaN,0.0,NaN,US,A,2014.0,US,A,2005.0,58017860.0,1.0
2694968,53873510,322442943,1.0,NaN,0.0,NaN,US,A,2004.0,HK,A,2007.0,35446125.0,1.0


In [97]:
# Computing the numbers of records that are recognized as "exact_twins" with family size of "one"
mask0_ = df_diff.is_in_1==1
mask1_ = df_diff.docdb_family_size==1
mask2_ = df_diff.exact_twins_1==1

df_ = df_diff[(mask0_)&(mask1_)&(mask2_)]
print('There are {:,} patents with family size of one, which are identified as exact twins!\n'.format(df_.shape[0]))

There are 82 patents with family size of one, which are identified as exact twins!



First, we take a look at `"439138526"-"442209040"` patent pairs, which has been identified as exact twins. 

Let's take a look at the first element, `439138526` patent, which has **docdb_family_id = 61898886**

In [36]:
%%bigquery 
SELECT 
    appln_id, 
    appln_nr, 
    appln_filing_year, 
    appln_auth, 
    docdb_family_id, 
    docdb_family_size
FROM `usptobias.patstat.tls201`
WHERE docdb_family_id=61898886

,appln_id,appln_nr,appln_filing_year,appln_auth,docdb_family_id,docdb_family_size
0,439138526,201313964854,2013,US,61898886,1


As it appears, there is only one member in this family, so this pair cannot be an exact twin!

So, let's take a look at the second member in this pair, which is **appln_id_2 = 442209040**

In [8]:
%%bigquery 
SELECT 
    appln_id, 
    appln_nr, 
    appln_filing_year, 
    appln_auth, 
    docdb_family_id, 
    docdb_family_size
FROM `usptobias.patstat.tls201`
WHERE appln_id=442209040

,appln_id,appln_nr,appln_filing_year,appln_auth,docdb_family_id,docdb_family_size
0,442209040,201213603107,2012,US,61898885,4


As we can see the `docdb_family_id` here is different from the docdb_family_id of patent with appln_id **439138526**

Checking for two other appln_id's of `family_size = 1` to see if it's the same case.

In [40]:
%%bigquery 
SELECT 
    appln_id, 
    appln_nr,
    docdb_family_id, 
    docdb_family_size
FROM `usptobias.patstat.tls201`
WHERE appln_id IN(470578509, 2301821)

,appln_id,appln_nr,docdb_family_id,docdb_family_size
0,2301821,7570298,25341142,9
1,470578509,201615185298,59966075,1


In [41]:
%%bigquery 
SELECT 
    appln_id, 
    appln_nr,
    docdb_family_id, 
    docdb_family_size
FROM `usptobias.patstat.tls201`
WHERE appln_id IN(364305866, 441053844)

,appln_id,appln_nr,docdb_family_id,docdb_family_size
0,441053844,201414563888,46198213,1
1,364305866,201113288721,61148141,1


So, for family size of `1` we can almost be sure that sould never get a pair and also they are not exact twins.

## 3.1.3. Problem 2 - Situation 2: family size of 2

Now, let's take a look at pairs of family size `2` (which on paper should be exact twins if the second member has `appln_auth` of 'US'

In [98]:
# Getting a list of patents with family size of one!
mask0_ = df_diff.docdb_family_size==2
mask1_ = df_diff.is_in_1==1
print('There are {:,} patents with family size of two!\n'.format(df_diff[(mask0_)&(mask1_)].shape[0]))
df_diff[(mask0_)&(mask1_)].head()

There are 50,078 patents with family size of two!



,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size
1284,47630882,19147245,1.0,NaN,1.0,NaN,US,A,2002.0,FR,A,1902.0,27804405.0,2.0
6081,53299069,43587097,1.0,NaN,1.0,NaN,US,A,2001.0,SE,A,1995.0,20399399.0,2.0
6916,48980593,42231938,1.0,NaN,1.0,NaN,US,A,2002.0,OA,A,1995.0,33304044.0,2.0
7520,54062163,43584981,1.0,NaN,1.0,NaN,US,A,2001.0,SE,A,1995.0,26662294.0,2.0
16042,52995516,20552408,1.0,NaN,1.0,NaN,US,A,2001.0,FR,A,1997.0,9506448.0,2.0


Let's query the `docdb_family_id` of index 5534, for an instance:

In [69]:
%%bigquery 
SELECT 
    appln_id, 
    appln_nr,
    appln_auth,
    docdb_family_id, 
    docdb_family_size
FROM `usptobias.patstat.tls201`
WHERE docdb_family_id=27804405

,appln_id,appln_nr,appln_auth,docdb_family_id,docdb_family_size
0,19147245,325790D,FR,27804405,2
1,47630882,10928102,US,27804405,2


And, as another try, let's query the family members of index 7512:

In [11]:
%%bigquery 
SELECT 
    appln_id, 
    appln_nr,
    appln_auth,
    docdb_family_id, 
    docdb_family_size
FROM `usptobias.patstat.tls201`
WHERE docdb_family_id=26662294

,appln_id,appln_nr,appln_auth,docdb_family_id,docdb_family_size
0,54062163,90147101,US,26662294,2
1,43584981,9501657,SE,26662294,2


In both cases, we can see that those are valid exact twins! Thus, we can say that in general, those with family size 2 should be recognized as exact twins.

Now, lets see if there is any patent with the family size of two in `df_twins_1` which are not identified as exact twins?

In [100]:
# Getting a list of patents with family size of one!
mask0_ = df_diff.docdb_family_size==2
mask1_ = df_diff.is_in_1==1
mask2_ = df_diff.exact_twins_1==0

df_ = df_diff[(mask0_)&(mask1_)&(mask2_)]

print('There are {:,} patents with family size of two, which are not identified as exact twins!\n'.format(df_.shape[0]))

df_diff[(mask0_)&(mask1_)&(mask2_)].head(4)

There are 3,151 patents with family size of two, which are not identified as exact twins!



,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size
49399,332590977,41490112,1.0,NaN,0.0,NaN,US,A,2010.0,NL,A,1999.0,26478258.0,2.0
56386,48963102,20614049,1.0,NaN,0.0,NaN,US,A,2002.0,FR,A,1999.0,26401552.0,2.0
58819,53002564,381216237,1.0,NaN,0.0,NaN,US,A,2001.0,YU,A,1999.0,29414618.0,2.0
59881,53250793,20606329,1.0,NaN,0.0,NaN,US,A,2001.0,FR,A,1999.0,26408653.0,2.0


Let's take a look at the family of index 40719:

## Look at the 

In [74]:
%%bigquery 
SELECT 
    appln_id, 
    appln_nr,
    appln_auth,
    docdb_family_id, 
    docdb_family_size
FROM `usptobias.patstat.tls201`
WHERE docdb_family_id=26478258

,appln_id,appln_nr,appln_auth,docdb_family_id,docdb_family_size
0,332590977,94447110,US,26478258,2
1,41490112,1012170,NL,26478258,2


So, indeed they are exact twins! but in `df_twins_1` they were mistakenly labeled as not exact twins!

But, why they are not identified in `df_twins_2`? (which is the table of my query results)

In [101]:
mask0_ = df_diff.is_in_1==1
mask1_ = df_diff.docdb_family_size==2
mask2_ = df_diff.appln_filing_year_2>=2001
mask3_ = df_diff.appln_filing_year_1>=2001
mask4_ = df_diff.appln_kind_2=='A '
print('There are {:,} patents which are of family size "2" and filed after 2001!\n'.format(df_diff[(mask0_)&(mask1_)&(mask2_)&(mask3_)&(mask4_)].shape[0]))

df_diff[(mask0_)&(mask1_)&(mask2_)&(mask3_)&(mask4_)].head(4)

There are 409 patents which are of family size "2" and filed after 2001!



,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size
105140,48982121,321219943,1.0,NaN,0.0,NaN,US,A,2002.0,AR,A,2002.0,59724211.0,2.0
105301,50204255,19038406,1.0,NaN,0.0,NaN,US,A,2003.0,FR,A,2002.0,39525420.0,2.0
105305,53512448,321219943,1.0,NaN,0.0,NaN,US,A,2004.0,AR,A,2002.0,59724211.0,2.0
111496,275113352,273499700,1.0,NaN,1.0,NaN,US,A,2009.0,NZ,A,2005.0,47326737.0,2.0


So, this is because of the conditions that I put when I was querying to produce the resulting table (which I considered only patents after 2001 and with kind of 'A' to be joined to 'USPatents' table)

## 3.1.3. Problem 2 - Situation 3: family size of more than 2!

Now, let's take a look at rows with family size of more than 2.

Here, it's tricky to identify what went wrong by just looking at family size and `docdb_family_id`. So, we need to also look at their `appln_auth`:

In [102]:
mask0_ = df_diff.is_in_1==1
mask1_ = df_diff.docdb_family_size>2

df_ = df_diff[(mask0_)&(mask1_)].sort_values(by=['docdb_family_id', 'appln_auth_2'])
print('There are {:,} patents with family size of "2"!\n'.format(df_.shape[0]))

df_.head(4)

There are 2,201,693 patents with family size of "2"!



,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size
342209,49159129,55210508,1.0,NaN,0.0,NaN,US,A,2005.0,AT,T,2002.0,3460581.0,14.0
349493,53138922,55210508,1.0,NaN,0.0,NaN,US,A,2003.0,AT,T,2002.0,3460581.0,14.0
326173,51363400,1014350,1.0,NaN,1.0,NaN,US,A,2004.0,AT,T,2001.0,3460621.0,10.0
78753,53465265,341631659,1.0,NaN,1.0,NaN,US,A,2001.0,AR,A,2000.0,3460791.0,3.0


Let's query `docdb_family_id` of "3460796" in index 93826, for an instance:

In [4]:
%%bigquery 
SELECT 
    appln_id, 
    appln_nr,
    appln_auth,
    docdb_family_id, 
    docdb_family_size
FROM `usptobias.patstat.tls201`
WHERE docdb_family_id=3460796

,appln_id,appln_nr,appln_auth,docdb_family_id,docdb_family_size
0,53524424,79933201,US,3460796,7
1,15805536,01104585,EP,3460796,7
2,41301084,PA01002421,MX,3460796,7
3,341631788,P000101005,AR,3460796,7
4,1609428,2485801,AU,3460796,7
5,3466059,0101054,BR,3460796,7
6,4703718,2339425,CA,3460796,7


We can see that the pairs in index "93826" should be identified as exact twins. The hypothesis here is that these were not identified in `df_Twins_2`, because of the conditions that I put in joining stage (joining on the rows which are filed after `2001` and has appln_kinf of `'A'`.

Let's investigate this hypothesis:

In [103]:
# Getting the results of records with family size of more than two, which are filed after "2001" 
# ...and has 'appln_kind' of "A"
mask0_ = df_diff.is_in_1==1
mask1_ = df_diff.docdb_family_size>2
mask2_ = df_diff.appln_filing_year_2>=2001
mask3_ = df_diff.appln_kind_2=='A '
mask4_ = df_diff.exact_twins_1==1

df_ = df_diff[(mask0_)&(mask1_)&(mask2_)&(mask3_)].sort_values(by=['docdb_family_id', 'appln_auth_2'])
print('There are {:,} patents with family size of more than "2", which are filed after 2001 and has kind="A"!\n'.format(df_.shape[0]))

df_.head(4)

There are 3,439 patents with family size of more than "2", which are filed after 2001 and has kind="A"!



,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size
4449326,50652197,54918846,1.0,NaN,0.0,NaN,US,A,2006.0,ZA,A,2003.0,3824803.0,18.0
4449373,50582213,54918846,1.0,NaN,0.0,NaN,US,A,2004.0,ZA,A,2003.0,3824803.0,18.0
4449376,53983070,54918846,1.0,NaN,0.0,NaN,US,A,2001.0,ZA,A,2003.0,3824803.0,18.0
4450009,267369187,54924126,1.0,NaN,0.0,NaN,US,A,2009.0,ZA,A,2004.0,3831972.0,16.0


These are the pairs that should be considered in my results `df_twins_1`. Let's investigate why they don't exist in the results, by looking at the index '4449435`, for an instance:

In [21]:
%%bigquery 
SELECT 
    appln_id, 
    appln_nr,
    appln_auth,
    appln_filing_year,
    docdb_family_id, 
    docdb_family_size
FROM `usptobias.patstat.tls201`
WHERE docdb_family_id=3824803

,appln_id,appln_nr,appln_auth,appln_filing_year,docdb_family_id,docdb_family_size
0,37969164,2002533875,JP,2001,3824803,18
1,15572402,200300461,EA,2001,3824803,18
2,42207933,52550801,NZ,2001,3824803,18
3,42234370,1200300114,OA,2001,3824803,18
4,50582213,39921304,US,2004,3824803,18
5,50652197,40810706,US,2006,3824803,18
6,24295012,15543301,IL,2001,3824803,18
7,53983070,88526801,US,2001,3824803,18
8,54918363,200302931,ZA,2003,3824803,18
9,411493285,20037005234,KR,2003,3824803,18


We see that the `appln_id_2` of the pair '4449435' is not in the same family! So, it's not a valid pair. Let's check the information of `appln_id_2 = 54918846` application:

In [22]:
%%bigquery 
SELECT 
    appln_id, 
    appln_nr,
    appln_auth,
    docdb_family_id, 
    docdb_family_size
FROM `usptobias.patstat.tls201`
WHERE appln_id=54918846

,appln_id,appln_nr,appln_auth,docdb_family_id,docdb_family_size
0,54918846,200303684,ZA,49356609,2


In [23]:
%%bigquery 
SELECT 
    appln_id, 
    appln_nr,
    appln_auth,
    docdb_family_id, 
    docdb_family_size
FROM `usptobias.patstat.tls201`
WHERE docdb_family_id=49356609

,appln_id,appln_nr,appln_auth,docdb_family_id,docdb_family_size
0,411500082,20037005253,KR,49356609,2
1,54918846,200303684,ZA,49356609,2


We see that "appln_id" of `54918846` is from another family id, which doesn't have a US member at all.

Let's try another pair. For instance index "4449866":

In [24]:
%%bigquery 
SELECT 
    appln_id, 
    appln_nr,
    appln_auth,
    appln_filing_year,
    docdb_family_id, 
    docdb_family_size
FROM `usptobias.patstat.tls201`
WHERE docdb_family_id=3831972

,appln_id,appln_nr,appln_auth,appln_filing_year,docdb_family_id,docdb_family_size
0,15972933,02766975,EP,2002,3831972,16
1,42212300,53221602,NZ,2002,3831972,16
2,38346817,2003533961,JP,2002,3831972,16
3,337470546,16129104,IL,2004,3831972,16
4,411807037,20047005136,KR,2002,3831972,16
5,1397373,0201338,WO,2002,3831972,16
6,2511493,PR814401,AU,2001,3831972,16
7,2546866,2002331460,AU,2002,3831972,16
8,2776593,2008201803,AU,2008,3831972,16
9,4838537,2462892,CA,2002,3831972,16


Again, the `appln_id_2 = 54924126` is not in the same family!

In [20]:
%%bigquery 
SELECT 
    appln_id_1,
    appln_id_2,
    appln_auth_1,
    appln_auth_2,
    appln_filing_year_1,
    appln_filing_year_2,
    docdb_family_id
    
FROM `usptobias.results_docdb.USOnDocdb`
WHERE appln_id_1=50652197

,appln_id_1,appln_id_2,appln_auth_1,appln_auth_2,appln_filing_year_1,appln_filing_year_2,docdb_family_id
0,50652197,42234370,US,OA,2006,2001,3824803
1,50652197,2446952,US,AU,2006,2001,3824803
2,50652197,4798020,US,CA,2006,2001,3824803
3,50652197,6763498,US,CN,2006,2001,3824803
4,50652197,37969164,US,JP,2006,2001,3824803
5,50652197,50582213,US,US,2006,2004,3824803
6,50652197,15572402,US,EA,2006,2001,3824803
7,50652197,3479206,US,BR,2006,2001,3824803
8,50652197,24295012,US,IL,2006,2001,3824803
9,50652197,411493285,US,KR,2006,2003,3824803


Now, let's look at **all of such pairs** and find out how many pairs with the family size of more than "2", have different family id!

In [30]:
# Loading the 'df_' dataframe as a temporary table on BigQuery
client = bigquery.Client()

dataset_id = 'results_docdb'
table_id = 'tmp_'
table_ref = client.dataset(dataset_id).table(table_id)

job = client.load_table_from_dataframe(df_, table_ref, location="US")
job.result()

In [105]:
%%bigquery df_result_
SELECT 
    appln_id_1,
    appln_id_2,
    t2.docdb_family_id AS docdb_family_id_1,
    t3.docdb_family_id AS docdb_family_id_2,
    t2.appln_auth AS appln_auth_1,
    t3.appln_auth AS appln_auth_2,
    t2.appln_filing_year AS filing_year_1,
    t3.appln_filing_year AS filing_year_2
FROM `usptobias.results_docdb.tmp_` AS t1
LEFT JOIN `usptobias.patstat.tls201` AS t2 ON t1.appln_id_1 = t2.appln_id
LEFT JOIN `usptobias.patstat.tls201` AS t3 ON t1.appln_id_2 = t3.appln_id

In [106]:
# Checking the result of the query
df_result_.head(3)

,appln_id_1,appln_id_2,docdb_family_id_1,docdb_family_id_2,appln_auth_1,appln_auth_2,filing_year_1,filing_year_2
0,421961759,445585550,50397350,50487158,US,AU,2014,2014
1,48461530,404895383,60407550,22660471,US,US,2002,2012
2,54183598,8319077,35787581,35599211,US,CN,2007,2005


In [107]:
# Specifying whether each rows have the same 'docdb_family_id' or not
df_result_['is_same_family'] = df_result_.apply(lambda x: 1 if x.iloc[2]==x.iloc[3] else 0, axis=1)
# Counting how many of the rows had the same 'docdb_family_id'
print('There are {} rows which have the same docdb_family_id number!\n'.format(df_result_['is_same_family'].sum()))
df_result_.head(4)

There are 4 rows which have the same docdb_family_id number!



,appln_id_1,appln_id_2,docdb_family_id_1,docdb_family_id_2,appln_auth_1,appln_auth_2,filing_year_1,filing_year_2,is_same_family
0,421961759,445585550,50397350,50487158,US,AU,2014,2014,0
1,48461530,404895383,60407550,22660471,US,US,2002,2012,0
2,54183598,8319077,35787581,35599211,US,CN,2007,2005,0
3,54373678,410485510,39473348,38896888,US,RU,2008,2007,0


We see that only **4 rows** out of 3,439 records has the same 'docdb_family_id', which means the rest are invalid pairs and should not exists in this list. 

But let's investigate these **4** records in more details:

In [48]:
# Let's investigate those rows with the same docdb_family_id
df_result_[df_result_.is_same_family==1]

,appln_id_1,appln_id_2,docdb_family_id_1,docdb_family_id_2,appln_auth_1,appln_auth_2,filing_year_1,filing_year_2,is_same_family
307,48175620,450949815,10865363,10865363,US,JP,2000,2015,1
689,48175620,415788155,10865363,10865363,US,JP,2000,2013,1
2196,48175620,274744143,10865363,10865363,US,AU,2000,2010,1
2438,48175620,334537538,10865363,10865363,US,JP,2000,2011,1


As we can see here, the first item in these pairs (i.e. appln_id_1) they all have `appln_filing_year` before **2001**! So, again the results in `df_twins_2` make sense.

## 3.2. Looking at the pairs that only exist in `"df_twins_2"` (and not in `df_twins_1`)


In [123]:
mask1_ = df_diff.is_in_2==1

# Getting rows in which 'df_twins_1' has an invalid appln_id
df_ = df_diff[(mask1_)]

print('There are {:,} rows which only exist in "df_twins_2" table!'.format(df_.shape[0]))
df_.head()

There are 2,216,439 rows which only exist in "df_twins_2" table!


,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size
101028,46003839,476498178,NaN,1.0,NaN,1.0,US,A,2005.0,SA,A,2001.0,8310615.0,14.0
101034,414785832,476307455,NaN,1.0,NaN,0.0,US,A,2013.0,EA,A,2001.0,27392237.0,10.0
101037,46823316,483358633,NaN,1.0,NaN,0.0,US,A,2005.0,SA,A,2001.0,7650138.0,30.0
101038,49750903,476504028,NaN,1.0,NaN,0.0,US,A,2003.0,SA,A,2001.0,27255657.0,31.0
101039,52091877,476307171,NaN,1.0,NaN,0.0,US,A,2006.0,EA,A,2001.0,9884904.0,21.0


### 3.2.1. Looking at invalid `appln_id` rows

In [109]:
#
mask1_ = df_diff.is_in_2==1
mask2_ = df_diff.appln_auth_2.isna()
mask3_ = df_diff.appln_auth_1.isna()

# Getting rows in which 'df_twins_1' has an invalid appln_id
df_invalid1 = df_diff[(mask1_)&((mask2_)|(mask3_))]

print('There are {:,} rows which has an invalid appln_id!'.format(df_invalid1.shape[0]))
df_invalid1.head()

There are 0 rows which has an invalid appln_id!


,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size


So, there are no rows with invalid values of `appln_id_1` or `appln_id_2`

## 3.1.2. Problem 2 - Situation 1: family size of 1

In [110]:
# Getting a list of patents with family size of "one"
mask0_ = df_diff.is_in_2==1
mask1_ = df_diff.docdb_family_size==1

df_ = df_diff[(mask0_)&(mask1_)]
print('There are {:,} patents with family size of one!'.format(df_.shape[0]))
df_.sort_values(by=['docdb_family_size']).head()

There are 0 patents with family size of one!


,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size


So, again no problem with having `appln_id`s of family size "One"

## 3.1.3. Problem 2 - Situation 2: family size of 2

In [111]:
# Getting a list of patents with family size of two!
mask0_ = df_diff.docdb_family_size==2
mask1_ = df_diff.is_in_2==1

df_ = df_diff[(mask0_)&(mask1_)]
print('There are {:,} patents with family size of two!\n'.format(df_.shape[0]))
df_.head()

There are 99,506 patents with family size of two!



,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size
101740,54055898,24154670,NaN,1.0,NaN,1.0,US,A,2001.0,IE,A,2001.0,26320297.0,2.0
102817,53655774,476307560,NaN,1.0,NaN,1.0,US,A,2001.0,EA,A,2001.0,27222369.0,2.0
102840,54055903,24154671,NaN,1.0,NaN,1.0,US,A,2001.0,IE,A,2001.0,26320295.0,2.0
103109,54130001,24154668,NaN,1.0,NaN,1.0,US,A,2001.0,IE,A,2001.0,26320301.0,2.0
103262,54055951,24154669,NaN,1.0,NaN,1.0,US,A,2001.0,IE,A,2001.0,26320296.0,2.0


We know that all of these cases should be considered as exact twins. Let's investigate that:

In [112]:
# Getting a list of patents with family size of two which are recognized as exact twins!
mask0_ = df_diff.docdb_family_size==2
mask1_ = df_diff.is_in_2==1
mask2_ = df_diff.exact_twins_2==1

df_ = df_diff[(mask0_)&(mask1_)&(mask2_)]
print('There are {:,} rows with family size of two, which has been identified as exact twins!\n'.format(df_.shape[0]))

There are 52,322 rows with family size of two, which has been identified as exact twins!



As we can see, there are lots of different rows which are note identified as exact twins. Let's investigate those:

In [113]:
# Getting a list of patents with family size of two which are not recognized as exact twins!
mask0_ = df_diff.docdb_family_size==2
mask1_ = df_diff.is_in_2==1
mask2_ = df_diff.exact_twins_2==0

df_ = df_diff[(mask0_)&(mask1_)&(mask2_)]
print('There are {:,} rows with family size of two, which has been identified as exact twins!\n'.format(df_.shape[0]))
df_.head()

There are 47,184 rows with family size of two, which has been identified as exact twins!



,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size
3865493,497158025,54208263,NaN,1.0,NaN,0.0,US,A,2015.0,US,A,2001.0,25462020.0,2.0
3865837,46249909,53791666,NaN,1.0,NaN,0.0,US,A,2005.0,US,A,2001.0,27394379.0,2.0
3866146,489086456,53288479,NaN,1.0,NaN,0.0,US,A,2004.0,US,A,2001.0,37914209.0,2.0
3866236,490948828,53680156,NaN,1.0,NaN,0.0,US,A,2017.0,US,A,2001.0,25249621.0,2.0
3866315,470758183,488320099,NaN,1.0,NaN,0.0,US,A,2008.0,US,A,2001.0,57136190.0,2.0


In [114]:
# Checking how many of these results has 'US' as their 'appln_auth_2'
mask0_ = df_diff.docdb_family_size==2
mask1_ = df_diff.is_in_2==1
mask2_ = df_diff.exact_twins_2==0
mask3_ = df_diff.appln_auth_2=='US'

df_ = df_diff[(mask0_)&(mask1_)&(mask2_)&(mask3_)]
print('There are {:,} rows with family size of two, which has been identified as exact twins!\n'.format(df_.shape[0]))

There are 47,184 rows with family size of two, which has been identified as exact twins!



As we can see, all of rows which has family size of two and are not recognized as exact twins, has "US" as their `appln_auth_2`, which is correct not to be identified as exact twins.

## 3.1.3. Problem 2 - Situation 3: family size of more than 2!

In [119]:
mask0_ = df_diff.is_in_2==1
mask1_ = df_diff.docdb_family_size>2
mask2_ = df_diff.docdb_family_id==1574492
df_ = df_diff[(mask0_)&(mask1_)].sort_values(by=['docdb_family_id', 'appln_auth_2'])
print('There are {:,} patents with family size of "2"!\n'.format(df_.shape[0]))

df_.head(7)

There are 2,116,933 patents with family size of "2"!



,appln_id_1,appln_id_2,is_in_1,is_in_2,exact_twins_1,exact_twins_2,appln_auth_1,appln_kind_1,appln_filing_year_1,appln_auth_2,appln_kind_2,appln_filing_year_2,docdb_family_id,docdb_family_size
671182,489501542,474746582,NaN,1.0,NaN,1.0,US,A,2015.0,AU,A,2015.0,1574492.0,8.0
809426,489501542,493462603,NaN,1.0,NaN,1.0,US,A,2015.0,BR,A,2015.0,1574492.0,8.0
2289036,489501542,441420240,NaN,1.0,NaN,1.0,US,A,2015.0,EP,A,2015.0,1574492.0,8.0
197119,489501542,447052740,NaN,1.0,NaN,1.0,US,A,2015.0,FR,A,2014.0,1574492.0,8.0
2999937,489501542,482003714,NaN,1.0,NaN,1.0,US,A,2015.0,JP,A,2015.0,1574492.0,8.0
3244939,489501542,476488278,NaN,1.0,NaN,1.0,US,A,2015.0,KR,A,2015.0,1574492.0,8.0
3188791,409579898,477035713,NaN,1.0,NaN,0.0,US,A,2012.0,KR,A,2001.0,3486087.0,39.0


Based on the query that I wrote, generally these pairs should be exact twins. Let's investigate that by considering two arbitrary pairs (e.g. index "809426" and "3188791"):

In [5]:
%%bigquery 
SELECT 
    appln_id, 
    appln_nr, 
    appln_filing_year, 
    appln_auth, 
    docdb_family_id, 
    docdb_family_size
FROM `usptobias.patstat.tls201`
WHERE docdb_family_id=1574492

,appln_id,appln_nr,appln_filing_year,appln_auth,docdb_family_id,docdb_family_size
0,441031337,2015062966,2015,WO,1574492,8
1,441420240,15727674,2015,EP,1574492,8
2,447052740,1401348,2014,FR,1574492,8
3,474746582,2015273600,2015,AU,1574492,8
4,476488278,20177001021,2015,KR,1574492,8
5,482003714,2017517404,2015,JP,1574492,8
6,489501542,201515318588,2015,US,1574492,8
7,493462603,112016029186,2015,BR,1574492,8


In [120]:
%%bigquery 
SELECT 
    appln_id, 
    appln_nr, 
    appln_filing_year, 
    appln_auth, 
    docdb_family_id, 
    docdb_family_size
FROM `usptobias.patstat.tls201`
WHERE docdb_family_id=3486087

,appln_id,appln_nr,appln_filing_year,appln_auth,docdb_family_id,docdb_family_size
0,477035713,20017010518,2001,KR,3486087,39
1,417905950,09154489,2000,PT,3486087,39
2,37188131,2000599878,2000,JP,3486087,39
3,13780653,50011116,2000,DE,3486087,39
4,15469995,00904677,2000,DK,3486087,39
5,15577348,P200100432,2000,EE,3486087,39
6,17642073,00904677,2000,ES,3486087,39
7,23873067,P20010681,2001,HR,3486087,39
8,23935695,0200394,2000,HU,3486087,39
9,24284597,14477700,2000,IL,3486087,39


And we can see that the pairs with index `809426` correctly identified as an exact twin, and index `3188791` correctly categorized as **not** an exact twin